Sept 6 assignment


In [ ]:
! pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=11b84c040d8b1f761225a79f59e8b392fe13e31deb36b47638401dc1b7e5ab04
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession




 # **Fitness Tracker Data**

In [ ]:
spark = SparkSession.builder.appName("FitnessTrackerAnalysis").getOrCreate()

In [ ]:
csv_file_path = "/content/sample_data/FitnessTrackerData.csv"

#Now you can read it with pyspark
fitness_df = spark.read.format("csv").option("header", "true").load(csv_file_path)
fitness_df.show()

+-------+----------+-----+--------+-----------+--------------+
|user_id|      date|steps|calories|distance_km|active_minutes|
+-------+----------+-----+--------+-----------+--------------+
|      1|2023-07-01|12000|     500|        8.5|            90|
|      2|2023-07-01| 8000|     350|        5.6|            60|
|      3|2023-07-01|15000|     600|       10.2|           120|
|      1|2023-07-02|11000|     480|        7.9|            85|
|      2|2023-07-02| 9000|     400|        6.2|            70|
|      3|2023-07-02|13000|     520|        9.0|           100|
|      1|2023-07-03|10000|     450|        7.1|            80|
|      2|2023-07-03| 7000|     320|        4.9|            55|
|      3|2023-07-03|16000|     620|       11.0|           130|
+-------+----------+-----+--------+-----------+--------------+



In [ ]:
#Find the Total Steps Taken by Each User
total_steps_by_user = fitness_df.groupBy("User_ID").agg(sum("steps").alias("Total_Steps"))
total_steps_by_user.show()

+-------+-----------+
|User_ID|Total_Steps|
+-------+-----------+
|      3|    44000.0|
|      1|    33000.0|
|      2|    24000.0|
+-------+-----------+



In [ ]:
 #Filter Days Where a User Burned More Than 500 Calories
high_calorie_days = fitness_df.filter(col("calories") > 500)
high_calorie_days.show()

+-------+----------+-----+--------+-----------+--------------+
|user_id|      date|steps|calories|distance_km|active_minutes|
+-------+----------+-----+--------+-----------+--------------+
|      3|2023-07-01|15000|     600|       10.2|           120|
|      3|2023-07-02|13000|     520|        9.0|           100|
|      3|2023-07-03|16000|     620|       11.0|           130|
+-------+----------+-----+--------+-----------+--------------+



In [ ]:
# 3. Calculate the Average Distance Traveled by Each User
from pyspark.sql.functions import round
avg_distance_df = fitness_df.groupBy("user_id").agg(round(avg("distance_km"),3).alias("avg_distance"))
avg_distance_df.show()



+-------+------------+
|user_id|avg_distance|
+-------+------------+
|      3|      10.067|
|      1|       7.833|
|      2|       5.567|
+-------+------------+



In [ ]:
# 4. dentify the Day with the Maximum Steps for Each User
from pyspark.sql.functions import max

max_steps_df = fitness_df.groupBy("user_id").agg(max("steps").alias("max_steps"), max("date").alias("max_steps_date"))
max_steps_df.show()



+-------+---------+--------------+
|user_id|max_steps|max_steps_date|
+-------+---------+--------------+
|      1|    12000|    2023-07-03|
|      2|     9000|    2023-07-03|
|      3|    16000|    2023-07-03|
+-------+---------+--------------+



In [ ]:
# 5. Find Users Who Were Active for More Than 100 Minutes on Any Day
active_users_df = fitness_df.filter(fitness_df.active_minutes > 100).select("user_id").distinct()
active_users_df.show()


+-------+
|user_id|
+-------+
|      3|
+-------+



In [ ]:
# 6. Calculate the Total Calories Burned per Day
from pyspark.sql.functions import sum

total_calories_per_day_df = fitness_df.groupBy("date").agg(sum("calories").alias("total_calories"))
total_calories_per_day_df.show()

+----------+--------------+
|      date|total_calories|
+----------+--------------+
|2023-07-03|        1390.0|
|2023-07-02|        1400.0|
|2023-07-01|        1450.0|
+----------+--------------+



In [ ]:
# 7. Calculate the Average Steps per Day
from pyspark.sql.functions import avg

avg_steps_per_day_df = fitness_df.groupBy("date").agg(round(avg("steps"),3).alias("avg_steps"))
avg_steps_per_day_df.show()


+----------+---------+
|      date|avg_steps|
+----------+---------+
|2023-07-03|  11000.0|
|2023-07-02|  11000.0|
|2023-07-01|11666.667|
+----------+---------+



In [ ]:
# 8. Rank Users by Total Distance Traveled
from pyspark.sql.functions import sum

total_distance_df = fitness_df.groupBy("user_id").agg(round(sum("distance_km"),2).alias("total_distance"))
ranked_df = total_distance_df.orderBy("total_distance", ascending=False)
ranked_df.show()


+-------+--------------+
|user_id|total_distance|
+-------+--------------+
|      3|          30.2|
|      1|          23.5|
|      2|          16.7|
+-------+--------------+



In [ ]:
# 9. Find the Most Active User by Total Active Minutes
from pyspark.sql.functions import sum

most_active_user_df = fitness_df.groupBy("user_id").agg(sum("active_minutes").alias("total_active_minutes"))
most_active_user_df.orderBy("total_active_minutes", ascending=False).limit(1).show()


+-------+--------------------+
|user_id|total_active_minutes|
+-------+--------------------+
|      3|               350.0|
+-------+--------------------+



In [ ]:
# 10. Create a New Column for Calories Burned per Kilometer
from pyspark.sql.functions import col
fitness_data = fitness_df.withColumn("calories_per_km", round(col("calories") / col("distance_km"),2))
fitness_data.show()

+-------+----------+-----+--------+-----------+--------------+---------------+
|user_id|      date|steps|calories|distance_km|active_minutes|calories_per_km|
+-------+----------+-----+--------+-----------+--------------+---------------+
|      1|2023-07-01|12000|     500|        8.5|            90|          58.82|
|      2|2023-07-01| 8000|     350|        5.6|            60|           62.5|
|      3|2023-07-01|15000|     600|       10.2|           120|          58.82|
|      1|2023-07-02|11000|     480|        7.9|            85|          60.76|
|      2|2023-07-02| 9000|     400|        6.2|            70|          64.52|
|      3|2023-07-02|13000|     520|        9.0|           100|          57.78|
|      1|2023-07-03|10000|     450|        7.1|            80|          63.38|
|      2|2023-07-03| 7000|     320|        4.9|            55|          65.31|
|      3|2023-07-03|16000|     620|       11.0|           130|          56.36|
+-------+----------+-----+--------+-----------+-----

 # **Book Sales Data**

In [ ]:

spark = SparkSession.builder.appName("BookSalesAnalysis").getOrCreate()

booksales_data = "/content/sample_data/BookSalesData.csv"

booksales_df = spark.read.csv(booksales_data, header=True, inferSchema=True)
booksales_df.show()

+-------+--------------------+-------------------+---------------+----------+--------+----------+
|sale_id|          book_title|             author|          genre|sale_price|quantity|      date|
+-------+--------------------+-------------------+---------------+----------+--------+----------+
|      1|The Catcher in th...|      J.D. Salinger|        Fiction|     15.99|       2|2023-01-05|
|      2|To Kill a Mocking...|         Harper Lee|        Fiction|     18.99|       1|2023-01-10|
|      3|            Becoming|     Michelle Obama|      Biography|      20.0|       3|2023-02-12|
|      4|             Sapiens|  Yuval Noah Harari|    Non-Fiction|      22.5|       1|2023-02-15|
|      5|            Educated|      Tara Westover|      Biography|     17.99|       2|2023-03-10|
|      6|    The Great Gatsby|F. Scott Fitzgerald|        Fiction|     10.99|       5|2023-03-15|
|      7|       Atomic Habits|        James Clear|      Self-Help|     16.99|       3|2023-04-01|
|      8|           

In [ ]:
# 1. Find Total Sales Revenue per Genre
book_sales_df = booksales_df.withColumn("total_sales", col("sale_price") * col("quantity"))
total_revenue_by_genre = book_sales_df.groupBy("genre").agg(round(sum("total_sales"),2).alias("total_revenue"))
total_revenue_by_genre.show()

+---------------+-------------+
|          genre|total_revenue|
+---------------+-------------+
|        Fiction|        135.9|
|      Self-Help|        68.97|
|Science Fiction|        25.99|
|    Non-Fiction|         22.5|
|      Biography|        95.98|
+---------------+-------------+



In [ ]:
# 2. Filter Books Sold in the "Fiction" Genre
fiction_books_df = booksales_df.filter(book_sales_df.genre == "Fiction")
fiction_books_df.show()


+-------+--------------------+-------------------+-------+----------+--------+----------+
|sale_id|          book_title|             author|  genre|sale_price|quantity|      date|
+-------+--------------------+-------------------+-------+----------+--------+----------+
|      1|The Catcher in th...|      J.D. Salinger|Fiction|     15.99|       2|2023-01-05|
|      2|To Kill a Mocking...|         Harper Lee|Fiction|     18.99|       1|2023-01-10|
|      6|    The Great Gatsby|F. Scott Fitzgerald|Fiction|     10.99|       5|2023-03-15|
|      9|                1984|      George Orwell|Fiction|     14.99|       2|2023-04-12|
+-------+--------------------+-------------------+-------+----------+--------+----------+



In [ ]:
# 3. Find the Book with the Highest Sale Price
highest_sale_price_df = booksales_df.orderBy("sale_price", ascending=False).limit(1)
highest_sale_price_df.show()


+-------+----------+-------------+---------------+----------+--------+----------+
|sale_id|book_title|       author|          genre|sale_price|quantity|      date|
+-------+----------+-------------+---------------+----------+--------+----------+
|      8|      Dune|Frank Herbert|Science Fiction|     25.99|       1|2023-04-10|
+-------+----------+-------------+---------------+----------+--------+----------+



In [ ]:
# 4. Calculate Total Quantity of Books Sold by Author
from pyspark.sql.functions import sum

total_quantity_by_author = booksales_df.groupBy("author").agg(sum("quantity").alias("total_quantity_sold"))
total_quantity_by_author.show()


+-------------------+-------------------+
|             author|total_quantity_sold|
+-------------------+-------------------+
|      Frank Herbert|                  1|
|         Harper Lee|                  1|
|      George Orwell|                  2|
|F. Scott Fitzgerald|                  5|
|     Michelle Obama|                  3|
|      J.D. Salinger|                  2|
|     Charles Duhigg|                  1|
|  Yuval Noah Harari|                  1|
|        James Clear|                  3|
|      Tara Westover|                  2|
+-------------------+-------------------+



In [ ]:
# 5. Identify Sales Transactions Worth More Than $50

high_value_sales = book_sales_df.filter(col("total_sales") > 50)
high_value_sales.show()

+-------+----------------+-------------------+---------+----------+--------+----------+-----------+
|sale_id|      book_title|             author|    genre|sale_price|quantity|      date|total_sales|
+-------+----------------+-------------------+---------+----------+--------+----------+-----------+
|      3|        Becoming|     Michelle Obama|Biography|      20.0|       3|2023-02-12|       60.0|
|      6|The Great Gatsby|F. Scott Fitzgerald|  Fiction|     10.99|       5|2023-03-15|      54.95|
|      7|   Atomic Habits|        James Clear|Self-Help|     16.99|       3|2023-04-01|      50.97|
+-------+----------------+-------------------+---------+----------+--------+----------+-----------+



In [ ]:
# 6. Find the Average Sale Price per Genre
avg_price_by_genre = book_sales_df.groupBy("genre").agg(round(avg("sale_price"),2).alias("avg_price"))
avg_price_by_genre.show()

+---------------+---------+
|          genre|avg_price|
+---------------+---------+
|        Fiction|    15.24|
|      Self-Help|    17.49|
|Science Fiction|    25.99|
|    Non-Fiction|     22.5|
|      Biography|    18.99|
+---------------+---------+



In [ ]:
# 7. Count the Number of Unique Authors in the Dataset
unique_authors_count = booksales_df.select("author").distinct().count()
print(f"Number of unique authors: {unique_authors_count}")


Number of unique authors: 10


In [ ]:
# 8. Find the Top 3 Best-Selling Books by Quantity
top_selling_books = booksales_df.groupBy("book_title").agg(sum("quantity").alias("total_quantity_sold"))
top_selling_books.orderBy("total_quantity_sold", ascending=False).limit(3).show()


+----------------+-------------------+
|      book_title|total_quantity_sold|
+----------------+-------------------+
|The Great Gatsby|                  5|
|   Atomic Habits|                  3|
|        Becoming|                  3|
+----------------+-------------------+



In [ ]:
# 9. Calculate Total Sales for Each Month
from pyspark.sql.functions import sum, month, col, expr


booksales_df_with_total_sales = booksales_df.withColumn("total_sales", expr("sale_price * quantity"))
total_sales_by_month = booksales_df_with_total_sales.groupBy(month(col("date")).alias("month")).agg(
    sum("total_sales").alias("total_sales")
)

total_sales_by_month.show()


+-----+-----------+
|month|total_sales|
+-----+-----------+
|    1|      50.97|
|    3|      90.93|
|    5|       18.0|
|    4|     106.94|
|    2|       82.5|
+-----+-----------+



In [ ]:
 # 10. Create a New Column for Total Sales Amount
booksales_df_with_total_sales = booksales_df.withColumn("total_sales", expr("sale_price * quantity"))
booksales_df_with_total_sales.show()


+-------+--------------------+-------------------+---------------+----------+--------+----------+-----------+
|sale_id|          book_title|             author|          genre|sale_price|quantity|      date|total_sales|
+-------+--------------------+-------------------+---------------+----------+--------+----------+-----------+
|      1|The Catcher in th...|      J.D. Salinger|        Fiction|     15.99|       2|2023-01-05|      31.98|
|      2|To Kill a Mocking...|         Harper Lee|        Fiction|     18.99|       1|2023-01-10|      18.99|
|      3|            Becoming|     Michelle Obama|      Biography|      20.0|       3|2023-02-12|       60.0|
|      4|             Sapiens|  Yuval Noah Harari|    Non-Fiction|      22.5|       1|2023-02-15|       22.5|
|      5|            Educated|      Tara Westover|      Biography|     17.99|       2|2023-03-10|      35.98|
|      6|    The Great Gatsby|F. Scott Fitzgerald|        Fiction|     10.99|       5|2023-03-15|      54.95|
|      7| 

 # **Food Delivery Orders**

In [ ]:
spark = SparkSession.builder.appName("/FoodDeliveryAnalysis").getOrCreate()

FoodDelivery_Data = "/content/sample_data/FoodDeliveryOrders.csv"

food_orders_df = spark.read.csv(booksales_data, header=True, inferSchema=True)
food_orders_df.show()

+----------+-----------+-------------+--------+--------------+---------+
|      date|       city|temperature_c|humidity|wind_speed_kph|condition|
+----------+-----------+-------------+--------+--------------+---------+
|2023-01-01|   New York|            5|      60|            20|   Cloudy|
|2023-01-01|Los Angeles|           15|      40|            10|    Sunny|
|2023-01-01|    Chicago|           -2|      75|            25|     Snow|
|2023-01-02|   New York|            3|      65|            15|     Rain|
|2023-01-02|Los Angeles|           18|      35|             8|    Sunny|
|2023-01-02|    Chicago|           -5|      80|            30|     Snow|
|2023-01-03|   New York|            6|      55|            22|    Sunny|
|2023-01-03|Los Angeles|           20|      38|            12|    Sunny|
|2023-01-03|    Chicago|           -1|      70|            18|   Cloudy|
+----------+-----------+-------------+--------+--------------+---------+



In [ ]:
# Calculate Total Revenue per Restaurant
total_revenue = food_orders_df.withColumn("total_revenue", food_orders_df.price * food_orders_df.quantity)
total_revenue.groupBy("restaurant_name").sum("total_revenue").show()


+---------------+------------------+
|restaurant_name|sum(total_revenue)|
+---------------+------------------+
|         Subway|              13.0|
|      Pizza Hut|             12.99|
|    Burger King|              6.99|
|            KFC|             35.96|
|       Domino's|             23.98|
|     McDonald's|20.950000000000003|
|      Starbucks|              13.5|
+---------------+------------------+



In [ ]:
# Find the Fastest Delivery:
Fastest_delivery = food_orders_df.orderBy("delivery_time_mins")

Fastest_delivery.show(1)


+--------+-----------+---------------+---------+--------+-----+------------------+----------+
|order_id|customer_id|restaurant_name|food_item|quantity|price|delivery_time_mins|order_date|
+--------+-----------+---------------+---------+--------+-----+------------------+----------+
|       6|        205|      Starbucks|   Coffee|       1|  4.5|                15|2023-06-18|
+--------+-----------+---------------+---------+--------+-----+------------------+----------+
only showing top 1 row



In [ ]:
# Calculate Average Delivery Time per Restaurant:
Avg_delivery = food_orders_df.groupBy("restaurant_name").avg("delivery_time_mins")
Avg_delivery.show()

+---------------+-----------------------+
|restaurant_name|avg(delivery_time_mins)|
+---------------+-----------------------+
|         Subway|                   20.0|
|      Pizza Hut|                   45.0|
|    Burger King|                   30.0|
|            KFC|                   25.0|
|       Domino's|                   40.0|
|     McDonald's|                   22.5|
|      Starbucks|                   17.5|
+---------------+-----------------------+



In [ ]:
# Filter Orders for a Specific Customer
Specific_Customer = food_orders_df.filter(food_orders_df.customer_id == 201)
Specific_Customer.show()


+--------+-----------+---------------+---------+--------+-----+------------------+----------+
|order_id|customer_id|restaurant_name|food_item|quantity|price|delivery_time_mins|order_date|
+--------+-----------+---------------+---------+--------+-----+------------------+----------+
|       1|        201|     McDonald's|   Burger|       2| 5.99|                30|2023-06-15|
|       4|        201|         Subway| Sandwich|       2|  6.5|                20|2023-06-17|
+--------+-----------+---------------+---------+--------+-----+------------------+----------+



In [ ]:
# Find Orders Where Total Amount Spent is Greater Than $20:
food_orders_df.filter((food_orders_df.price * food_orders_df.quantity) > 20).show()


+--------+-----------+---------------+-------------+--------+-----+------------------+----------+
|order_id|customer_id|restaurant_name|    food_item|quantity|price|delivery_time_mins|order_date|
+--------+-----------+---------------+-------------+--------+-----+------------------+----------+
|       3|        203|            KFC|Fried Chicken|       3| 8.99|                25|2023-06-17|
|       5|        204|       Domino's|        Pizza|       2|11.99|                40|2023-06-18|
+--------+-----------+---------------+-------------+--------+-----+------------------+----------+



In [ ]:
# Calculate the Total Quantity of Each Food Item Sold:
total_quantity = food_orders_df.groupBy("food_item").sum("quantity")
total_quantity.show()


+-------------+-------------+
|    food_item|sum(quantity)|
+-------------+-------------+
|        Fries|            3|
|       Burger|            3|
|     Sandwich|            2|
|        Pizza|            3|
|       Coffee|            3|
|Fried Chicken|            4|
+-------------+-------------+



In [ ]:
# Find the Top 3 Most Popular Restaurants by Number of Orders:
top_three_restaurants = food_orders_df.groupBy("restaurant_name").count().orderBy("count", ascending=False)
top_three_restaurants .show(3)


+---------------+-----+
|restaurant_name|count|
+---------------+-----+
|            KFC|    2|
|     McDonald's|    2|
|      Starbucks|    2|
+---------------+-----+
only showing top 3 rows



In [ ]:
# Calculate Total Revenue per Day:
Total_Revenue_per_Day = food_orders_df.withColumn("total_revenue", food_orders_df.price * food_orders_df.quantity).groupBy("order_date").sum("total_revenue")
Total_Revenue_per_Day.show()

+----------+------------------+
|order_date|sum(total_revenue)|
+----------+------------------+
|2023-06-18|             28.48|
|2023-06-16|             12.99|
|2023-06-15|             11.98|
|2023-06-17|             39.97|
|2023-06-20|             15.99|
|2023-06-19|             17.96|
+----------+------------------+



In [ ]:
# Find the Longest Delivery Time for Each Restaurant
food_orders_df.groupBy("restaurant_name").max("delivery_time_mins").show()


+---------------+-----------------------+
|restaurant_name|max(delivery_time_mins)|
+---------------+-----------------------+
|         Subway|                     20|
|      Pizza Hut|                     45|
|    Burger King|                     30|
|            KFC|                     25|
|       Domino's|                     40|
|     McDonald's|                     30|
|      Starbucks|                     20|
+---------------+-----------------------+



In [ ]:
# Create a New Column for Total Order Value
food_orders_df.withColumn("total_order_value", food_orders_df.price * food_orders_df.quantity).show()


+--------+-----------+---------------+-------------+--------+-----+------------------+----------+-----------------+
|order_id|customer_id|restaurant_name|    food_item|quantity|price|delivery_time_mins|order_date|total_order_value|
+--------+-----------+---------------+-------------+--------+-----+------------------+----------+-----------------+
|       1|        201|     McDonald's|       Burger|       2| 5.99|                30|2023-06-15|            11.98|
|       2|        202|      Pizza Hut|        Pizza|       1|12.99|                45|2023-06-16|            12.99|
|       3|        203|            KFC|Fried Chicken|       3| 8.99|                25|2023-06-17|            26.97|
|       4|        201|         Subway|     Sandwich|       2|  6.5|                20|2023-06-17|             13.0|
|       5|        204|       Domino's|        Pizza|       2|11.99|                40|2023-06-18|            23.98|
|       6|        205|      Starbucks|       Coffee|       1|  4.5|     

# **Weather Data**

In [ ]:
spark = SparkSession.builder.appName("/WeatherDataAnalysis").getOrCreate()

weatherAnalysis_data = "/content/sample_data/WeatherData.csv"

weather_data = spark.read.csv(booksales_data, header=True, inferSchema=True)
weather_data.show()

+----------+-----------+-------------+--------+--------------+---------+
|      date|       city|temperature_c|humidity|wind_speed_kph|condition|
+----------+-----------+-------------+--------+--------------+---------+
|2023-01-01|   New York|            5|      60|            20|   Cloudy|
|2023-01-01|Los Angeles|           15|      40|            10|    Sunny|
|2023-01-01|    Chicago|           -2|      75|            25|     Snow|
|2023-01-02|   New York|            3|      65|            15|     Rain|
|2023-01-02|Los Angeles|           18|      35|             8|    Sunny|
|2023-01-02|    Chicago|           -5|      80|            30|     Snow|
|2023-01-03|   New York|            6|      55|            22|    Sunny|
|2023-01-03|Los Angeles|           20|      38|            12|    Sunny|
|2023-01-03|    Chicago|           -1|      70|            18|   Cloudy|
+----------+-----------+-------------+--------+--------------+---------+



In [ ]:
# Find the Average Temperature for Each City

Avg_Temperature = weather_data.groupBy("city").avg("temperature_c")
Avg_Temperature.show()


+-----------+-------------------+
|       city| avg(temperature_c)|
+-----------+-------------------+
|Los Angeles| 17.666666666666668|
|    Chicago|-2.6666666666666665|
|   New York|  4.666666666666667|
+-----------+-------------------+



In [ ]:
# Filter Days with Temperature Below Freezing

Freezing_temp_days = weather_data.filter("temperature_c < 0")
Freezing_temp_days.show()


+----------+-------+-------------+--------+--------------+---------+
|      date|   city|temperature_c|humidity|wind_speed_kph|condition|
+----------+-------+-------------+--------+--------------+---------+
|2023-01-01|Chicago|           -2|      75|            25|     Snow|
|2023-01-02|Chicago|           -5|      80|            30|     Snow|
|2023-01-03|Chicago|           -1|      70|            18|   Cloudy|
+----------+-------+-------------+--------+--------------+---------+



In [ ]:
# Find the City with the Highest Wind Speed on a Specific Day
weather_data.filter("date == '2023-01-02'").orderBy("wind_speed_kph", ascending=False).show(1)


+----------+-------+-------------+--------+--------------+---------+
|      date|   city|temperature_c|humidity|wind_speed_kph|condition|
+----------+-------+-------------+--------+--------------+---------+
|2023-01-02|Chicago|           -5|      80|            30|     Snow|
+----------+-------+-------------+--------+--------------+---------+
only showing top 1 row



In [ ]:
# Calculate the Total Number of Days with Rainy Weather
Rainy_Weather = weather_data.filter("condition == 'Rain'")
Rainy_Weather.count()


1

In [ ]:
# Calculate the Average Humidity for Each Weather Condition
Avg_Humidity = weather_data.groupBy("condition").avg("humidity")
Avg_Humidity.show()


+---------+-------------+
|condition|avg(humidity)|
+---------+-------------+
|   Cloudy|         65.0|
|    Sunny|         42.0|
|     Snow|         77.5|
|     Rain|         65.0|
+---------+-------------+



In [ ]:
# Find the Hottest Day in Each City
Hottest_Day = weather_data.groupBy("city").agg({"temperature_c": "max"})
Hottest_Day.show()


+-----------+------------------+
|       city|max(temperature_c)|
+-----------+------------------+
|Los Angeles|                20|
|    Chicago|                -1|
|   New York|                 6|
+-----------+------------------+



In [ ]:
 # Identify Cities That Experienced Snow
Snow_city = weather_data.filter("condition == 'Snow'").select("city").distinct()
Snow_city.show()


+-------+
|   city|
+-------+
|Chicago|
+-------+



In [ ]:
# Calculate the Average Wind Speed for Days When the Condition was Sunny
weather_data.filter("condition == 'Sunny'").agg({"wind_speed_kph": "avg"}).show()


+-------------------+
|avg(wind_speed_kph)|
+-------------------+
|               13.0|
+-------------------+



In [ ]:
# Find the Coldest Day Across All Cities
Coldest_day = weather_data.orderBy("temperature_c")
Coldest_day.show(1)


+----------+-------+-------------+--------+--------------+---------+
|      date|   city|temperature_c|humidity|wind_speed_kph|condition|
+----------+-------+-------------+--------+--------------+---------+
|2023-01-02|Chicago|           -5|      80|            30|     Snow|
+----------+-------+-------------+--------+--------------+---------+
only showing top 1 row



In [ ]:
 # Create a New Column for Wind Chill
from pyspark.sql.functions import col, pow
weather_data_with_wind_chill = weather_data.withColumn(
    "wind_chill",
    13.12 + 0.6215 * col("temperature_c") - 11.37 * pow(col("wind_speed_kph"), 0.16) +
    0.3965 * col("temperature_c") * pow(col("wind_speed_kph"), 0.16)
)

weather_data_with_wind_chill.show()



+----------+-----------+-------------+--------+--------------+---------+-------------------+
|      date|       city|temperature_c|humidity|wind_speed_kph|condition|         wind_chill|
+----------+-----------+-------------+--------+--------------+---------+-------------------+
|2023-01-01|   New York|            5|      60|            20|   Cloudy| 1.0669572525115663|
|2023-01-01|Los Angeles|           15|      40|            10|    Sunny| 14.604602843130213|
|2023-01-01|    Chicago|           -2|      75|            25|     Snow| -8.479874917414646|
|2023-01-02|   New York|            3|      65|            15|     Rain|-0.7170927775232809|
|2023-01-02|Los Angeles|           18|      35|             8|    Sunny| 18.403050060338735|
|2023-01-02|    Chicago|           -5|      80|            30|     Snow|-12.996724811921073|
|2023-01-03|   New York|            6|      55|            22|    Sunny| 2.1056686374397366|
|2023-01-03|Los Angeles|           20|      38|            12|    Sunn

# **Airline Flight Data**

In [ ]:
spark = SparkSession.builder.appName("AirlinesAnalysis").getOrCreate()
csv_file_path = "/content/sample_data/AirlineFlightData.csv"

#Now you can read it with pyspark
airline_data = spark.read.format("csv").option("header", "true").load(csv_file_path)
airline_data.show()

+---------+---------+-------------+------+-----------+--------------+------------+---------+-----------------+----------+
|flight_id|  airline|flight_number|origin|destination|departure_time|arrival_time|delay_min|distance_traveled|      date|
+---------+---------+-------------+------+-----------+--------------+------------+---------+-----------------+----------+
|        1|    Delta|        DL123|   JFK|        LAX|         08:00|       11:00|       30|             3970|2023-07-01|
|        2|   United|        UA456|   SFO|        ORD|         09:30|       15:00|       45|             2960|2023-07-01|
|        3|Southwest|        SW789|   DAL|        ATL|         06:00|       08:30|        0|             1150|2023-07-01|
|        4|    Delta|        DL124|   LAX|        JFK|         12:00|       20:00|       20|             3970|2023-07-02|
|        5| American|        AA101|   MIA|        DEN|         07:00|       10:00|       15|             2770|2023-07-02|
|        6|   United|   

In [ ]:
# Find the Total Distance Traveled by Each Airline
Total_Distance_traveled = airline_data.groupBy("airline").agg({"distance_traveled": "sum"})
Total_Distance_traveled.show()

+---------+----------------------+
|  airline|sum(distance_traveled)|
+---------+----------------------+
|    Delta|               11840.0|
|   United|                5920.0|
|  JetBlue|                4180.0|
|Southwest|                2300.0|
| American|                5540.0|
+---------+----------------------+



In [ ]:
# Filter Flights with Delays Greater than 30 Minutes
Flight_delay = airline_data.filter("delay_min > 30")
Flight_delay.show()


+---------+-------+-------------+------+-----------+--------------+------------+---------+-----------------+----------+
|flight_id|airline|flight_number|origin|destination|departure_time|arrival_time|delay_min|distance_traveled|      date|
+---------+-------+-------------+------+-----------+--------------+------------+---------+-----------------+----------+
|        2| United|        UA456|   SFO|        ORD|         09:30|       15:00|       45|             2960|2023-07-01|
+---------+-------+-------------+------+-----------+--------------+------------+---------+-----------------+----------+



In [ ]:
 # Find the Flight with the Longest Distance
Longest_Distance_flight = airline_data.orderBy("distance_traveled", ascending=False)
Longest_Distance_flight.show(1)


+---------+-------+-------------+------+-----------+--------------+------------+---------+-----------------+----------+
|flight_id|airline|flight_number|origin|destination|departure_time|arrival_time|delay_min|distance_traveled|      date|
+---------+-------+-------------+------+-----------+--------------+------------+---------+-----------------+----------+
|        7|JetBlue|        JB302|   BOS|        LAX|         06:30|       09:45|       10|             4180|2023-07-03|
+---------+-------+-------------+------+-----------+--------------+------------+---------+-----------------+----------+
only showing top 1 row



In [ ]:
# Calculate the Average Delay Time for Each Airline
Avg_Delay = airline_data.groupBy("airline").agg({"delay_min": "avg"})
Avg_Delay.show()


+---------+------------------+
|  airline|    avg(delay_min)|
+---------+------------------+
|    Delta|16.666666666666668|
|   United|              22.5|
|  JetBlue|              10.0|
|Southwest|               2.5|
| American|              20.0|
+---------+------------------+



In [ ]:
# Identify Flights That Were Not Delayed
airline_data.filter("delay_min == 0").show()

+---------+---------+-------------+------+-----------+--------------+------------+---------+-----------------+----------+
|flight_id|  airline|flight_number|origin|destination|departure_time|arrival_time|delay_min|distance_traveled|      date|
+---------+---------+-------------+------+-----------+--------------+------------+---------+-----------------+----------+
|        3|Southwest|        SW789|   DAL|        ATL|         06:00|       08:30|        0|             1150|2023-07-01|
|        6|   United|        UA457|   ORD|        SFO|         11:00|       14:30|        0|             2960|2023-07-02|
|       10|    Delta|        DL125|   JFK|        SEA|         13:00|       17:00|        0|             3900|2023-07-04|
+---------+---------+-------------+------+-----------+--------------+------------+---------+-----------------+----------+



In [ ]:
# Find the Top 3 Most Frequent Routes
Frequent_Routes = airline_data.groupBy("origin", "destination").count().orderBy("count", ascending=False)
Frequent_Routes.show(3)


+------+-----------+-----+
|origin|destination|count|
+------+-----------+-----+
|   SFO|        ORD|    1|
|   LAX|        JFK|    1|
|   DEN|        MIA|    1|
+------+-----------+-----+
only showing top 3 rows



In [ ]:
# Calculate the Total Number of Flights per Day
Total_flights_perday = airline_data.groupBy("date").count()
Total_flights_perday.show()


+----------+-----+
|      date|count|
+----------+-----+
|2023-07-04|    1|
|2023-07-03|    3|
|2023-07-02|    3|
|2023-07-01|    3|
+----------+-----+



In [ ]:
# Find the Airline with the Most Flights
Airline_with_most_Flights = airline_data.groupBy("airline").count().orderBy("count", ascending=False)
Airline_with_most_Flights .show(1)


+-------+-----+
|airline|count|
+-------+-----+
|  Delta|    3|
+-------+-----+
only showing top 1 row



In [ ]:
# Calculate the Average Flight Distance per Day
airline_data.groupBy("date").agg({"distance_traveled": "avg"}).show()


+----------+----------------------+
|      date|avg(distance_traveled)|
+----------+----------------------+
|2023-07-04|                3900.0|
|2023-07-03|                2700.0|
|2023-07-02|    3233.3333333333335|
|2023-07-01|    2693.3333333333335|
+----------+----------------------+



In [ ]:
# Create a New Column for On-Time Status
from pyspark.sql.functions import when

airline_data = airline_data.withColumn("on_time", when(airline_data.delay_min == 0, True).otherwise(False))
airline_data.show()


+---------+---------+-------------+------+-----------+--------------+------------+---------+-----------------+----------+-------+
|flight_id|  airline|flight_number|origin|destination|departure_time|arrival_time|delay_min|distance_traveled|      date|on_time|
+---------+---------+-------------+------+-----------+--------------+------------+---------+-----------------+----------+-------+
|        1|    Delta|        DL123|   JFK|        LAX|         08:00|       11:00|       30|             3970|2023-07-01|  false|
|        2|   United|        UA456|   SFO|        ORD|         09:30|       15:00|       45|             2960|2023-07-01|  false|
|        3|Southwest|        SW789|   DAL|        ATL|         06:00|       08:30|        0|             1150|2023-07-01|   true|
|        4|    Delta|        DL124|   LAX|        JFK|         12:00|       20:00|       20|             3970|2023-07-02|  false|
|        5| American|        AA101|   MIA|        DEN|         07:00|       10:00|       1